In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tensorflow as tf
import sys
sys.path.append('../bmpc_shared_scripts/refinement_transfer_learning')
sys.path.append('../bmpc_shared_scripts/oktoberfest_interface')
from oktoberfest_interface import load_keras_model, process_dataset
from automatic_rl_tl import AutomaticRlTlTraining, AutomaticRlTlTrainingConfig

2024-07-27 13:26:43.382341: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 13:26:43.382396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 13:26:43.383635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 13:26:43.391156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-27 13:26:47.248314: W tensorflow/compiler/tf2


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ['HF_HOME'] = '/cmnfs/proj/bmpc_dlomix/datasets'
os.environ['HF_DATASETS_CACHE'] = '/cmnfs/proj/bmpc_dlomix/datasets/hf_cache'

num_proc = 16
os.environ["OMP_NUM_THREADS"] = f"{num_proc}"
os.environ["TF_NUM_INTRAOP_THREADS"] = f"{num_proc}"
os.environ["TF_NUM_INTEROP_THREADS"] = f"{num_proc}"

tf.config.threading.set_inter_op_parallelism_threads(num_proc)
tf.config.threading.set_intra_op_parallelism_threads(num_proc)

In [4]:
# small dataset
parquet_path = '/cmnfs/data/proteomics/Prosit_PTMs/21PTMs/Kmod_Formyl.parquet'
modifications = ['K[UNIMOD:122]']
# large single ptm
# parquet_path= '/cmnfs/data/proteomics/Prosit_PTMs/TUM_mod_monomethyl.parquet'
# modifications = ['K[UNIMOD:34]', 'R[UNIMOD:34]']
ion_types = ['y', 'b']

In [5]:
# load baseline model
model = load_keras_model('baseline')
model.summary()

2024-07-27 13:28:44.447371: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7505 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1


Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  464       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [6]:
dataset = process_dataset(
    parquet_file_path=parquet_path,
    model=model,
    modifications=modifications,
    ion_types=ion_types,
    label_column='intensities_raw',
    val_ratio=0.2
)


            There are new tokens in the dataset, which are not supported by the loaded model.
            Either load a different model or transfer learning needs to be done.
            
Casting the dataset: 100%|██████████| 1793/1793 [00:00<00:00, 3920.04 examples/s]


In [7]:
config = AutomaticRlTlTrainingConfig(
    dataset=dataset,
    baseline_model=model,
    use_wandb=True
)

In [8]:
trainer = AutomaticRlTlTraining(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: finnkap (mapra_dlomix). Use `wandb login --relogin` to force relogin


[embedding layer]  model and dataset modifications do not match
[embedding layer]  can reuse old embedding weights
[regressor layer]  matching ion types


In [9]:
new_model = trainer.train()

2024-07-27 13:29:06.237485: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


29/29 [==============================] - 4s 13ms/step - loss: 0.7064 - masked_pearson_correlation_distance: 0.5959
validation loss: 0.7064418196678162, pearson distance: 0.5959383249282837
Epoch 1/10554


2024-07-27 13:29:13.283062: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd80c379810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-27 13:29:13.283096: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-07-27 13:29:13.290597: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1722086953.407018  217079 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


113/113 [==============================] - 9s 32ms/step - loss: 0.7110 - masked_pearson_correlation_distance: 0.6067 - val_loss: 0.7064 - val_masked_pearson_correlation_distance: 0.5959
Epoch 2/10554
113/113 [==============================] - 3s 24ms/step - loss: 0.7114 - masked_pearson_correlation_distance: 0.6062 - val_loss: 0.7064 - val_masked_pearson_correlation_distance: 0.5959
Epoch 3/10554
113/113 [==============================] - 3s 24ms/step - loss: 0.7106 - masked_pearson_correlation_distance: 0.6053 - val_loss: 0.7064 - val_masked_pearson_correlation_distance: 0.5959
Epoch 4/10554
113/113 [==============================] - 3s 24ms/step - loss: 0.7101 - masked_pearson_correlation_distance: 0.6058 - val_loss: 0.7064 - val_masked_pearson_correlation_distance: 0.5959
Epoch 5/10554
113/113 [==============================] - 3s 24ms/step - loss: 0.7099 - masked_pearson_correlation_distance: 0.6052 - val_loss: 0.7063 - val_masked_pearson_correlation_distance: 0.5958
Epoch 6/10554


epoch,▁▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇▁▁▂▂▃▃▃▄▄▅▅▅▆▆▇▇██▁▁▂▂▃▃
epoch_total,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
es_avg_change,████▇▇▇▆▅▅▄▄▃▃▇▇▇▆▆▆▆▆▅▄▄▃▂▁▁▂██████████
es_curr_change,█████▇▆▅▅▄▄▄▄▄█▇▇▇▇▇▇▆▆▄▃▂▁▁▆███████████
freeze_embedding_layer,▁▁▁
freeze_inner_layers,█▁▁
freeze_old_embedding_weights,█▁▁
freeze_old_regressor_weights,▁▁▁
freeze_regressor_layer,█▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▂▃▅█▁▁▁▁▁▁▁▁▁▁▁▁▂▃▄███▂▂▂▂▂▁
loss,██████▇▇▆▆▆▆▆▆▅▅▆▅▆▅▆▅▅▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁
